In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K

## Classification and Imputation using Auto-Encoder

The idea is to train a dense neural network as an auto-encoder to fill in missing values.


## Outputs
Creates output files in the "predictions" directory.
Imputes as much as possible all NaNs in both train and test files and adds predictions.

**IMPORTANT**: Preidictions in the train set were created by the Auto-Encoder!!


In [2]:
# Load data
train = pd.read_csv('data/trainSet.txt')
test = pd.read_csv('data/testSet.txt')

In [3]:
# definde columns of interest
cols_out = ['PatientID', 'ImageFile', 'Hospital', 'Prognosis']
binary_vars = ['RespiratoryFailure', 'Sex', 'Cough', 'DifficultyInBreathing', 'CardiovascularDisease']
impute_cols = [i for i in train.columns if i not in cols_out]

## Autoencoder should work!

In [4]:
def create_model():
    
    regularizer = keras.regularizers.l1_l2(l1=0, l2=0.0015)
    
    inputs = keras.Input(shape=(16,))
    x = keras.layers.Dense(16, activation='selu')(inputs)
    # build a little auto-encoder
    x = keras.layers.Dense(100, activation='selu', kernel_regularizer=regularizer, name='expand')(x)
    x = keras.layers.Dropout(0.5)(x)
    # bottleneck
    x = keras.layers.Dense(4, activation='selu', kernel_regularizer=regularizer, name='bottleneck')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(100, activation='selu', kernel_regularizer=regularizer, name='reconstruct')(x)
    x = keras.layers.Dropout(0.5)(x)
    
    out1 = keras.layers.Dense(16, activation='sigmoid', name='imputing')(x)
    out2 = keras.layers.Dense(1, activation='sigmoid', name='classifying')(x)
    
    model = keras.Model(inputs, {'impute': out1, 'classify':out2})
    
    return model

In [5]:
# From https://github.com/keras-team/keras/issues/7065
# Calculate MSE only on known values
def masked_mse(mask_value=-1):
    def f(y_true, y_pred):
        mask_true = K.cast(K.not_equal(y_true, mask_value), K.floatx())
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sum(masked_squared_error, axis=-1) / K.sum(mask_true, axis=-1)
        return masked_mse * 100

    return f


# Validate imputation by removing one nan extra per column
def imputer_validation(valid_set):
    """
    Function to remove extra fields from data for validation.
    """
    fields_removed = valid_set.copy()
    imputer_mask = np.zeros(valid_set.shape)
    for n, row in enumerate(fields_removed):
        non_nan = np.where(row != -1)[0]
        set_nan = np.random.choice(non_nan)
        fields_removed[n, set_nan] = -1
        imputer_mask[n, set_nan] = True
    
    return fields_removed, imputer_mask


# helper to calculate scores
def calculate_score(x_true, x_pred, errorfun='mse', scaler=None):
    if scaler:
        x_true = scaler.inverse_transform(x_true.copy())
        x_pred = scaler.inverse_transform(x_pred.copy())
        x_true = np.clip(x_true, a_min=-1, a_max=None)
        x_pred = np.clip(x_pred, a_min=-1, a_max=None)
    if errorfun == 'mse':
        score = mean_squared_error(x_true, x_pred)
    elif errorfun == 'acc':
        score = accuracy_score(x_true, x_pred > 0.5)
    elif errorfun == "masked_mse":
        score = masked_mse(-1)(x_true, x_pred)
        score = np.mean(score)
    
    return score


def post_processing(original_df, predicted_values, classify, scaler=None):
    # Post processing
    nan_mask = np.isnan(original_df)
    previous_values = original_df.copy().values
    imputed_thing = np.zeros(previous_values.shape)

    # Rescale values and add to empty matrix
    if scaler is not None:
        predicted_values = MM.inverse_transform(predicted_values)
    
    imputed_thing[nan_mask] = predicted_values[nan_mask]
    # Clip negatives
    imputed_thing = np.clip(imputed_thing, a_min=0, a_max=None) 
    # Add known values
    imputed_thing[nan_mask==False] = previous_values[nan_mask==False]
    
    # Add classification and column names, binarize binary values
    df_out = pd.DataFrame(imputed_thing, columns=original_df.columns)
    df_out[binary_vars] = (df_out[binary_vars] > 0.5) * 1.0
    df_out['Prognosis'] = classify > 0.5
    
    return df_out

In [6]:
# some global vars
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
OPTIMIZER = keras.optimizers.Adam(lr=0.001)
CALLBACKS = [early_stopping]
LOSS = {'impute': masked_mse(), 'classify': 'binary_crossentropy'}

In [7]:
# Preprare data for training
train_clean = train[impute_cols]
train_y = train['Prognosis'] == 'MILD'

In [8]:
accs, mses0, mses1, imps0, imps1, trues, preds, indices, hosp_val = [], [], [], [], [], [], [], [], []

train_mask = np.isnan(train_clean).values

for hosp in np.unique(train.Hospital):
    model = create_model()
    model.compile(optimizer=OPTIMIZER, 
                  loss=LOSS,
                  metrics={'impute': 'mse', 'classify': 'accuracy'})
    MM = MinMaxScaler(clip=True)
    
    tr = np.where(train.Hospital != hosp)[0]
    vl = np.where(train.Hospital == hosp)[0]

    # Remove Nans
    x_train = train_clean.iloc[tr].copy().fillna(0)
    x_valid = train_clean.iloc[vl].copy().fillna(0)
    # Define y-values
    y_train = train_y.iloc[tr]
    y_valid = train_y.iloc[vl]
    # Scale data from 0 to 1
    x_train = MM.fit_transform(x_train)
    x_valid = MM.transform(x_valid)
    
    # Set former nan values to -1
    x_train[train_mask[tr]] = -1
    x_valid[train_mask[vl]] = -1
    
    # Fit Model
    model.fit(x_train, {'impute':x_train, 'classify': y_train}, verbose=0, 
              callbacks=CALLBACKS, validation_split=0.15,
             epochs=250)
    
    # Predict the things
    prediction = model.predict(x_valid)
    preds.append(MM.inverse_transform(prediction['impute']))

    pred_val, pred_y = prediction['impute'], prediction['classify']
    
    
    # Returns metrics, weirdly
    _, _, _, train_acc, train_mse = model.evaluate(x_train, 
                                                   {'impute':x_train, 
                                                    'classify': y_train}, 
                                                   verbose=0)
    
    # Set previous NAN to -1
    pred_val[train_mask[vl]] = -1
    x_valid[train_mask[vl]] = -1
    
    # imputer validation:
    x_impute, impute_mask = imputer_validation(x_valid)
    
    imputer_predict = model.predict(x_impute)['impute']
    imputer_predict[impute_mask==False] = -1
    
    validation_mask = x_valid.copy()
    validation_mask[impute_mask==False] = -1
    
    # imputer error
    imp0 = calculate_score(validation_mask, 
                            imputer_predict, errorfun='masked_mse')
    imp1 = calculate_score(validation_mask, 
                            imputer_predict, errorfun='masked_mse', scaler=MM)
    
    # Keepingt track
    imps0.append(imp0)
    imps1.append(imp1)
    mses0.append(calculate_score(x_valid, pred_val, scaler=MM, errorfun='masked_mse'))
    mses1.append(calculate_score(x_valid, pred_val, errorfun='masked_mse'))
    accs.append(calculate_score(y_valid, pred_y, errorfun='acc'))
    trues.append(MM.inverse_transform(x_valid))
    preds.append(MM.inverse_transform(prediction['impute']))
    print(f"Split {hosp}, with validation n = {x_valid.shape[0]}: \nRescaled MSE - {mses0[-1]:4.2f} " +
          f"MSE - {mses1[-1]:4.2f} " +
          f"Acc - {accs[-1]:4.2f}\n" +
          f"Impute MSE - {imps0[-1]:4.2f} " +
          f"Impute MSE scaled - {imps1[-1]:4.2f}")
          
    print(f"Train - MSE {train_mse:4.3f}, ACC {train_acc:4.3f}")
    
    indices.append(vl)
    hosp_val.append([hosp] * len(vl))
    print("==============================================")

Split B, with validation n = 104: 
Rescaled MSE - 78367448.00 MSE - 1.65 Acc - 0.68
Impute MSE - 13.38 Impute MSE scaled - 404964.81
Train - MSE 0.557, ACC 0.640
Split C, with validation n = 31: 
Rescaled MSE - 9070708.00 MSE - 0.91 Acc - 0.55
Impute MSE - 13.41 Impute MSE scaled - 3955.22
Train - MSE 0.515, ACC 0.651
Split D, with validation n = 139: 
Rescaled MSE - 115400168.00 MSE - 4.82 Acc - 0.71
Impute MSE - 11.36 Impute MSE scaled - 1395718528.00
Train - MSE 0.548, ACC 0.630
Split E, with validation n = 101: 
Rescaled MSE - 169587.83 MSE - 0.90 Acc - 0.63
Impute MSE - 11.93 Impute MSE scaled - 105284.49
Train - MSE 0.523, ACC 0.659
Split F, with validation n = 488: 
Rescaled MSE - 94116584.00 MSE - 3.05 Acc - 0.61
Impute MSE - 15.00 Impute MSE scaled - 6779072.00
Train - MSE 0.491, ACC 0.693


# Final Training and Creating Test Set

In [ ]:
model = create_model()
model.compile(optimizer=OPTIMIZER, 
                  loss=LOSS,
                  metrics={'impute': 'mae', 'classify': 'accuracy'})

MM = MinMaxScaler(clip=True)

# Transform Data:
x_train = train_clean.copy().fillna(0)
# Define y-values
y_train = train_y
# Scale data from 0 to 1
x_train = MM.fit_transform(x_train)
# Set former nan values to -1
x_train[train_mask] = -1


model.fit(x_train, {'impute':x_train, 'classify': y_train}, verbose=1, callbacks=CALLBACKS, 
          validation_split=0.15, epochs=250)

Epoch 1/250
23/23 [==============================] - 0s 14ms/step - loss: 13.7470 - classifying_loss: 0.8038 - imputing_loss: 12.8807 - classifying_accuracy: 0.4884 - imputing_mae: 0.5360 - val_loss: 9.7187 - val_classifying_loss: 0.7955 - val_imputing_loss: 8.8624 - val_classifying_accuracy: 0.4385 - val_imputing_mae: 0.5062
Epoch 2/250
23/23 [==============================] - 0s 3ms/step - loss: 10.2831 - classifying_loss: 0.7800 - imputing_loss: 9.4442 - classifying_accuracy: 0.5061 - imputing_mae: 0.4851 - val_loss: 8.2479 - val_classifying_loss: 0.6892 - val_imputing_loss: 7.5016 - val_classifying_accuracy: 0.5462 - val_imputing_mae: 0.4968
Epoch 3/250
23/23 [==============================] - 0s 3ms/step - loss: 9.1038 - classifying_loss: 0.7230 - imputing_loss: 8.3250 - classifying_accuracy: 0.5402 - imputing_mae: 0.4711 - val_loss: 7.3759 - val_classifying_loss: 0.6909 - val_imputing_loss: 6.6309 - val_classifying_accuracy: 0.5385 - val_imputing_mae: 0.4886
Epoch 4/250
23/23 [==

23/23 [==============================] - 0s 3ms/step - loss: 5.2935 - classifying_loss: 0.6702 - imputing_loss: 4.5779 - classifying_accuracy: 0.5921 - imputing_mae: 0.4133 - val_loss: 2.5142 - val_classifying_loss: 0.6559 - val_imputing_loss: 1.8131 - val_classifying_accuracy: 0.6385 - val_imputing_mae: 0.4225
Epoch 26/250
23/23 [==============================] - 0s 3ms/step - loss: 5.4309 - classifying_loss: 0.6769 - imputing_loss: 4.7089 - classifying_accuracy: 0.5812 - imputing_mae: 0.4138 - val_loss: 2.5239 - val_classifying_loss: 0.6571 - val_imputing_loss: 1.8218 - val_classifying_accuracy: 0.6385 - val_imputing_mae: 0.4225
Epoch 27/250
23/23 [==============================] - 0s 3ms/step - loss: 5.2964 - classifying_loss: 0.6665 - imputing_loss: 4.5850 - classifying_accuracy: 0.6030 - imputing_mae: 0.4123 - val_loss: 2.4767 - val_classifying_loss: 0.6522 - val_imputing_loss: 1.7797 - val_classifying_accuracy: 0.6385 - val_imputing_mae: 0.4213
Epoch 28/250
23/23 [===============

23/23 [==============================] - 0s 3ms/step - loss: 5.2748 - classifying_loss: 0.6724 - imputing_loss: 4.5613 - classifying_accuracy: 0.5962 - imputing_mae: 0.4099 - val_loss: 2.2440 - val_classifying_loss: 0.6519 - val_imputing_loss: 1.5510 - val_classifying_accuracy: 0.6462 - val_imputing_mae: 0.4160
Epoch 51/250
23/23 [==============================] - 0s 3ms/step - loss: 5.0927 - classifying_loss: 0.6758 - imputing_loss: 4.3759 - classifying_accuracy: 0.5894 - imputing_mae: 0.4082 - val_loss: 2.2341 - val_classifying_loss: 0.6523 - val_imputing_loss: 1.5409 - val_classifying_accuracy: 0.6385 - val_imputing_mae: 0.4149
Epoch 52/250
23/23 [==============================] - 0s 3ms/step - loss: 5.2422 - classifying_loss: 0.6707 - imputing_loss: 4.5308 - classifying_accuracy: 0.6071 - imputing_mae: 0.4100 - val_loss: 2.2137 - val_classifying_loss: 0.6564 - val_imputing_loss: 1.5166 - val_classifying_accuracy: 0.6462 - val_imputing_mae: 0.4151
Epoch 53/250
23/23 [===============

23/23 [==============================] - 0s 3ms/step - loss: 4.9482 - classifying_loss: 0.6662 - imputing_loss: 4.2428 - classifying_accuracy: 0.6221 - imputing_mae: 0.4057 - val_loss: 2.1023 - val_classifying_loss: 0.6557 - val_imputing_loss: 1.4075 - val_classifying_accuracy: 0.6385 - val_imputing_mae: 0.4095
Epoch 76/250
23/23 [==============================] - 0s 3ms/step - loss: 5.1747 - classifying_loss: 0.6658 - imputing_loss: 4.4698 - classifying_accuracy: 0.5907 - imputing_mae: 0.4068 - val_loss: 2.0737 - val_classifying_loss: 0.6542 - val_imputing_loss: 1.3804 - val_classifying_accuracy: 0.6385 - val_imputing_mae: 0.4095
Epoch 77/250
23/23 [==============================] - 0s 3ms/step - loss: 5.0796 - classifying_loss: 0.6564 - imputing_loss: 4.3842 - classifying_accuracy: 0.6289 - imputing_mae: 0.4064 - val_loss: 2.0673 - val_classifying_loss: 0.6557 - val_imputing_loss: 1.3727 - val_classifying_accuracy: 0.6538 - val_imputing_mae: 0.4094
Epoch 78/250
23/23 [===============

23/23 [==============================] - 0s 3ms/step - loss: 4.8503 - classifying_loss: 0.6566 - imputing_loss: 4.1554 - classifying_accuracy: 0.5948 - imputing_mae: 0.4036 - val_loss: 1.9978 - val_classifying_loss: 0.6604 - val_imputing_loss: 1.2990 - val_classifying_accuracy: 0.6154 - val_imputing_mae: 0.4071
Epoch 101/250
23/23 [==============================] - 0s 3ms/step - loss: 5.0361 - classifying_loss: 0.6650 - imputing_loss: 4.3327 - classifying_accuracy: 0.5880 - imputing_mae: 0.4066 - val_loss: 2.0323 - val_classifying_loss: 0.6577 - val_imputing_loss: 1.3361 - val_classifying_accuracy: 0.6385 - val_imputing_mae: 0.4078
Epoch 102/250
23/23 [==============================] - 0s 3ms/step - loss: 4.9996 - classifying_loss: 0.6598 - imputing_loss: 4.3014 - classifying_accuracy: 0.6003 - imputing_mae: 0.4058 - val_loss: 2.0115 - val_classifying_loss: 0.6550 - val_imputing_loss: 1.3181 - val_classifying_accuracy: 0.6385 - val_imputing_mae: 0.4077
Epoch 103/250
 1/23 [>...........

In [ ]:
train_predictions = model.predict(x_train)
train_imputations = train_predictions['impute']
train_classify = train_predictions['classify']

train_out = post_processing(train_clean, train_imputations, train_classify, MM)

In [ ]:
# Add missing columns
train_out[['PatientID', 'ImageFile', 'Hospital']] = train[['PatientID', 'ImageFile', 'Hospital']]
# Change order to original
train_out = train_out[train.columns]

train_out.to_csv('predictions/simon_TRAIN_impute_classify.csv', index=None)

In [ ]:
test_clean = test[impute_cols] 
x_test = test_clean.copy().fillna(0)
x_test = MM.transform(x_test)
x_test[np.isnan(test_clean)] = -1

In [ ]:
predictions = model.predict(x_test)

In [ ]:
# Predidctions
imputations = predictions['impute']
classify = predictions['classify']

In [ ]:
test_out = post_processing(test_clean, imputations, classify, MM)

test_out.loc[test_out.Prognosis==True, 'Prognosis'] = "MILD"
test_out.loc[test_out.Prognosis==False, 'Prognosis'] = "SEVERE"

In [ ]:
# Add missing columns
test_out[['PatientID', 'ImageFile', 'Hospital']] = test[['PatientID', 'ImageFile', 'Hospital']]
# Change order to original
test_out = test_out[test.columns]

In [ ]:
# Save to file
test_out.to_csv('predictions/simon_TEST_impute_classify.csv', index=False)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96)